In [48]:
# Loading required libraries
import csv
import sys
import pandas as pd
import os
import glob
import itertools
import numpy as np
from collections import Counter
import copy
import math



# Loading trace : Needs to expanded into 4K chunks
path = r'C:\Users\cchak\Desktop\Data_ECML\VDI Traces\selected_few'
all_files = glob.glob(os.path.join(path, "synthetic_dataprep_A_deathtime_added.csv"))

f = all_files[0]  # Change the file name as required
print("Working with file " + str(f))
cols = ['IO_num','LBA','Deathtime_RWI']
df = pd.read_csv(f,engine='python',skiprows =1,header=None,na_values=['-1'], index_col=False)
df.columns = cols
df['Deathtime_RWI'] = df['Deathtime_RWI'].replace(np.NaN, -1)
lba_list = df['LBA'].tolist()
deathtime_list = df['Deathtime_RWI'].tolist()
print("Min LBA in the dataset :", min(lba_list))
print("Max LBA in the dataset :", max(lba_list))
print("Number of unique LBAs in the data :",len(Counter(df['LBA'])))
print("Number of IO Accesses :",len(df))

Working with file C:\Users\cchak\Desktop\Data_ECML\VDI Traces\selected_few\synthetic_dataprep_A_deathtime_added.csv
Min LBA in the dataset : 0
Max LBA in the dataset : 1216608
Number of unique LBAs in the data : 1216609
Number of IO Accesses : 12166090


In [49]:
# SSD specifications
page_size = 4096
page_per_block = 64                          # Hyperparameter 
over_provisioning_ratio = 0.3 
GB = 1024*1024*1024
SSD_size_GB = 4.66 * GB
ssd_capacity = SSD_size_GB                    # Hyperparameter                     
LOG_PAGE_PER_BLOCK = int(math.log(page_per_block,2))


# Make the block,page and physical addresses for normal and Overprovisioned capacity
page_addresses = []
block_addresses = []
block_placement = 0
start_counter = -1
block_addresses.append(0)

while(start_counter < (ssd_capacity/page_size) - page_size):
    start_counter = start_counter + 1
    page_addresses.append(int(start_counter))
    if(block_placement >= page_per_block):
        block_addresses.append(int(start_counter))
        block_placement = 0

    block_placement = block_placement + 1

free_list_block = copy.deepcopy(block_addresses)
free_list_page = copy.deepcopy(page_addresses)


print("Total number of Blocks created: ", len(block_addresses))
print("Total number of Pages created:  ", len(page_addresses))

Total number of Blocks created:  19024
Total number of Pages created:   1217497


In [50]:
# Defining block_structure
# Block structure : Dictionary  
# Key: Block_address_start    
#Value: [start_lba,invalid_pages,valid_bitmap,write_ptr,death_time,priority,death_time_original]

block_struct = {}
for x in free_list_block:
    start_lba = x
    write_ptr=0
    invalid_pages=0
    death_time = 0
    valid_bitmap = []
    priority = False
    death_time_original = 0
    for x in range(page_per_block):
        valid_bitmap.append(False)

    segment = [start_lba,invalid_pages,valid_bitmap,write_ptr,death_time,priority,death_time_original]
    block_struct[start_lba]=segment
    
print(str(len(block_struct)) + " 4K blocks Initialized" )

19024 4K blocks Initialized


In [51]:


def invalidate_lba(lba):
    prev = L2P[lba]
    prev_block = (prev >> LOG_PAGE_PER_BLOCK)*page_per_block
    prev_page = prev % page_per_block
    block_details = block_struct[prev_block]                                 # Getting block details
    block_struct[prev_block][1] = block_struct[prev_block][1] - 1            # Decreasing invalid pages
    block_struct[prev_block][2][prev_page] = False                          # Setting bitmap to False
    L2P.pop(lba)

    

    
#map LBA to phys
def map_lba(lba,deathtime, block_IO_burst):
    # Finding which block to add the LBA
    block_map={}
    found = False
    for x in block_IO_burst:
        # If the death Time has passed, make it priority 
        if(block_struct[x][5] == True):
            block_select = x
            found = True 
        else:
            block_map[x]= block_struct[x][4]
    # Finding the block with closest death time 
    if(found !=True):
        delta = max(deathtime_range_list)*100
        keys = list(block_map.keys())
        block_select = -1
        for x in keys:
            tmp = abs(block_map[x] - deathtime)
            if(tmp < delta):
                delta = tmp
                block_select = x
    
    # Block Found, now updating block          
    phys_addr = block_struct[block_select][0] + (block_struct[block_select][3])
    L2P[lba] = phys_addr
    P2L[phys_addr] = lba   
    block_struct[block_select][2][block_struct[block_select][3]] = True                # Setting Bitmap
    block_struct[block_select][1] = block_struct[block_select][1] + 1               # Increasing invalid pages
    block_struct[block_select][3] = block_struct[block_select][3] + 1               # Increasing Write pointer

    

#check if we need to close/open block. Do not perform GC if we are already
def check_GC (block_IO_burst, in_gc):
    for x in block_IO_burst:
        # If block is full, close block and reset death time
        if(block_struct[x][3] >= page_per_block):
            death_time = block_struct[x][6]               # Copying original death time to be set in the new block
            closed_blocks.append(x)                       # Adding to closed list
            block_IO_burst.remove(x)                    
            new_block = free_list_block.pop(0)            #  Requesting a new block
            block_IO_burst.append(new_block)              # Adding to open blocks
            block_struct[new_block][4] = death_time       # Setting death time counter of the new block
            block_struct[new_block][6] = death_time       # Setting death time of the new block
    
    if(len(free_list_block) == 0):
        print("FAIL WHILE DOING GC, RAN OUT OF BLOCKS") 
    
    # Checking if GC is needed
    elif (len(free_list_block) <= GC_THRESHOLD):
        # Checking if GC is already going on
        if(in_gc != True):
            print("GC called")
            in_gc = do_greedy_gc(block_IO_burst,in_gc) 
    return block_IO_burst




def do_greedy_gc(block_IO_burst,in_gc):
#     print(counter)
    in_gc = True
    gc_writes = 0 
    for x in closed_blocks:
        min_val = float('inf')              
        if (block_struct[x][1] < min_val):
            min_val = block_struct[x][1]
        gc_blk = x
        # For each closed block, check phys_addr: If valid bitmap is True (data is valid), copy to OP capacity
    for pg in range(page_per_block):
        #figure out the logical addresses for all phys pages in the gc block
        phys_addr = block_struct[gc_blk][0] + pg
        # Updating P2L
        if (phys_addr in P2L):
            gc_lba = P2L[phys_addr]
            P2L.pop(phys_addr)            
        # Updating L2P
        if gc_lba in L2P:
            invalidate_lba(gc_lba)
        # Checking for valid bitmap
        prev_block = (phys_addr >> LOG_PAGE_PER_BLOCK)*page_per_block
        prev_page = phys_addr % page_per_block
        bitmap = block_struct[prev_block][2][prev_page]
        # If valid bitmap is True (data is valid), copy to OP capacity, increase GC writes
        if (bitmap != False):
            gc_writes = gc_writes + 1
            #check if we need to get a new block
            block_IO_burst = check_GC(block_IO_burst,in_gc)
            #move the gc'ed block t-o a new location
            map_lba(lba,death_time,block_IO_burst)
            block_IO_burst = decrease_death_time(block_IO_burst)  
    
    gc_file = r'C:\Users\cchak\Desktop\Data_ECML\VDI Traces\selected_few\GC_write_stats_FTL_modified_A.txt'
    file_object = open(gc_file, 'a')
    # Append new GC writes
    file_object.write(str(gc_writes) + "\n")
    # Close the file
    file_object.close()
    # Reseting the GC blk and add to free_block_list
    block_struct[gc_blk]= [gc_blk,invalid_pages,valid_bitmap,write_ptr,death_time,priority,death_time_original]
    closed_blocks.remove(gc_blk)
    free_list_block.append(gc_blk)
#     print("Number of GC writes in this cycle",gc_writes)
    in_gc = False
    return in_gc

def decrease_death_time(block_IO_burst):
    # Decreasing Death Time for each block
    for x in block_IO_burst:
        # If death time passed, make the block priority 
        # Priority means: All subsequent IOs will be added to this block here until full
        if (block_struct[x][4] <= 0):
            block_struct[x][5] = True
        else:
            block_struct[x][4] = block_struct[x][4] - 1  
    return block_IO_burst

# Printing Block Details
def print_cur_open_blocks(block_IO_burst):
    print("Block details")
    for x in block_IO_burst:
        print(block_struct[x])
    print()

In [52]:
# Setting global parameters
# Initalizing Starting Free Blocks..
global gc_writes
global in_gc 


GC_THRESHOLD = 0.2*len(block_addresses)
num_cur_blocks_open = 5        # Hyperparameter

L2P = {}
P2L = {}
closed_blocks = []
cur_blocks_open = []
lba_burst = []
deathtime_range_list = []
interval = float(100/num_cur_blocks_open)
gc_writes = 0
in_gc = False
death_time_passed = []

for x in range(num_cur_blocks_open):
    deathtime_range_list.append(int(np.percentile(deathtime_list, (x+1)*interval)))


block_IO_burst = []
death_time_ranges = []
print("Initalizing Starting Free Blocks...")
for x in range(num_cur_blocks_open):
    block_num = free_list_block.pop(0)
    block_IO_burst.append(block_num)
    block_struct[block_num][4] = deathtime_range_list[x]
    block_struct[block_num][6] = deathtime_range_list[x]



Initalizing Starting Free Blocks...


In [ ]:
counter = 0
print("Starting Trace..!")
while(counter < len(lba_list)):
    if(counter >100000 and counter%100000==0):
        print("Percentage completed in (%)  :", (counter/len(lba_list))*100)
    lba= int(lba_list[counter])
    death_time = int(deathtime_list[counter])
    if lba in L2P:
        invalidate_lba(lba)
    block_IO_burst = check_GC(block_IO_burst,in_gc)
    map_lba(lba,death_time,block_IO_burst)
    block_IO_burst = decrease_death_time(block_IO_burst)
    counter = counter + 1
print("Done")

Starting Trace..!
Percentage completed in (%)  : 1.6439135334359682
Percentage completed in (%)  : 2.4658703001539526
Percentage completed in (%)  : 3.2878270668719365
Percentage completed in (%)  : 4.109783833589921
Percentage completed in (%)  : 4.931740600307905
Percentage completed in (%)  : 5.753697367025889
Percentage completed in (%)  : 6.575654133743873
Percentage completed in (%)  : 7.397610900461858
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC call

GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called


GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called


GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called


GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called


...

GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called


GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called


GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called


GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called


GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called
GC called


In [ ]:
print('Done...Sanity Check complete!!')